In [1]:
from openpyxl import Workbook, load_workbook
import pandas as pd
from openpyxl.utils.dataframe import dataframe_to_rows as df_to_row
import numpy as np

In [6]:
from szp_funcs import path_to_data, print_df, to_double

In [3]:
months = ['jan', 'feb', 'mar', 'apr']

In [72]:
def load_month(file):
    df = pd.read_excel(path_to_data + file + '.xlsx')
    df['stv'] = df.apply(lambda row: to_double(row, 'stv'), axis=1)
    df = df[df.type.isin(day_types[:2])]
    sums = df[['inn', 'snils', 'sum']].groupby(['inn', 'snils']).sum()
    sums = pd.merge(sums.reset_index(), df[(df.status != 'Увольнение')], how='left', on=['inn', 'snils']).drop(columns='sum_y').rename(columns={'sum_x': 'sum'})
    sums = sums[sums.type == day_types[0]]
    sums.columns = ['inn', 'snils'] + [file + '_' + i for i in sums.columns][2:]
    return sums

In [79]:
def load_month_by_type(file, type):
    df = pd.read_excel(path_to_data + file + '.xlsx')
    df['stv'] = df.apply(lambda row: to_double(row, 'stv'), axis=1)
    df = df[df.type == day_types[type]]
    sums = df[['inn', 'snils', 'sum']].groupby(['inn', 'snils']).sum()
    sums = pd.merge(sums.reset_index(), df[(df.status != 'Увольнение')], how='left', on=['inn', 'snils']).drop(columns='sum_y').rename(columns={'sum_x': 'sum'})
    sums = sums[sums.type == day_types[type]]
    sums.columns = ['inn', 'snils'] + [file + '_' + i for i in sums.columns][2:]
    return sums

In [82]:
res_osn = custom_create_res(months, load_month_by_type, 0)
res_vnut = custom_create_res(months, load_month_by_type, 1)
res_vnesh = custom_create_res(months, load_month_by_type, 2)

In [83]:
print_df(res_osn, 'Свод основное до ' + months[-1])
print_df(res_vnut, 'Свод внутреннее до ' + months[-1])
print_df(res_vnesh, 'Свод внешнее до ' + months[-1])

In [81]:
def custom_create_res(months, func, type = -1):
    if type == -1:
        res = 0
        for i in range(len(months)):
            if i == 0:
                res = func(months[i])
            else:
                res = pd.merge(res, func(months[i]), how='outer', on=['inn', 'snils'])
        return res
    res = 0
    for i in range(len(months)):
        if i == 0:
            res = func(months[i], type)
        else:
            res = pd.merge(res, func(months[i], type), how='outer', on=['inn', 'snils'])
    return res

In [76]:
res = custom_create_res(months, load_month)

In [77]:
res

,inn,snils,jan_sum,jan_job,jan_day,jan_type,jan_stv,jan_status,jan_status_pref,feb_sum,...,mar_status_pref,apr_sum,apr_job,apr_day,apr_type,apr_stv,apr_status,apr_status_pref,apr_wday,apr_nday
0,5003021368,001-171-362 75,51468.76,Помощник воспитателя,0.0,Основное место работы,1.0,Работа,Работа,41289.47,...,Работа,58500.00,Помощник воспитателя,1.0,Основное место работы,1.0,Работа,Работа,21.0,21.0
1,5003021368,001-191-281 85,106626.88,Учитель,1.0,Основное место работы,1.0,Работа,Работа,139753.92,...,Работа,105890.88,Учитель,1.0,Основное место работы,1.0,Работа,Работа,21.0,21.0
2,5003021368,001-244-107 65,109394.87,Учитель,1.0,Основное место работы,1.0,Работа,Работа,109541.13,...,Работа,98522.15,Учитель,1.0,Основное место работы,1.0,Работа,Работа,21.0,21.0
3,5003021368,001-263-234 77,74520.00,Воспитатель,1.0,Основное место работы,1.0,Работа,Работа,81178.00,...,Работа,77794.00,Воспитатель,1.0,Основное место работы,1.0,Работа,Работа,21.0,21.0
4,5003021368,001-322-594 80,83013.52,Специалист,1.0,Основное место работы,1.0,Работа,Работа,75815.00,...,Работа,75815.00,Специалист,1.0,Основное место работы,1.0,Работа,Работа,21.0,21.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175916,9718071371,143-168-262 49,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,107526.00,Бухгалтер,1.0,Основное место работы,1.0,Работа,Работа,21.0,21.0
175917,9718071371,145-089-428 75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,48035.62,Бухгалтер,1.0,Основное место работы,1.0,Работа,NaN,13.0,13.0
175918,9718071371,145-624-332 54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,299871.00,Начальник отдела,1.0,Основное место работы,1.0,Работа,Работа,21.0,21.0
175919,9718071371,158-325-680 86,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,60000.00,Учитель,0.0,Основное место работы,1.0,Работа,Работа,NaN,NaN


In [78]:
print_df(res, 'Свод по начисл до ' + months[-1])

In [73]:
feb = load_month('feb')

In [70]:
feb.columns = ['inn', 'snils'] + ['feb_' + i for i in feb.columns][2:]

In [74]:
feb

,inn,snils,feb_sum,feb_job,feb_day,feb_type,feb_stv,feb_status,feb_status_pref
0,5003021368,001-171-362 75,41289.47,Помощник воспитателя,0.0,Основное место работы,1.0,Болезнь,Болезнь
1,5003021368,001-191-281 85,139753.92,Учитель,1.0,Основное место работы,1.0,Работа,Работа
2,5003021368,001-244-107 65,109541.13,Учитель,1.0,Основное место работы,1.0,Работа,Работа
3,5003021368,001-263-234 77,81178.00,Воспитатель,1.0,Основное место работы,1.0,Работа,Работа
4,5003021368,001-322-594 80,75815.00,Специалист,1.0,Основное место работы,1.0,Работа,Работа
...,...,...,...,...,...,...,...,...,...
189354,9718071371,199-418-719 35,131067.00,Бухгалтер,1.0,Основное место работы,1.0,Работа,Работа
189355,9718071371,203-134-432 96,50000.00,Учитель,0.0,Основное место работы,1.0,Работа,Работа
189356,9718071371,205-412-630 13,199010.00,Начальник группы,1.0,Основное место работы,1.0,Работа,Работа
189357,9718071371,205-678-005 60,24504.00,Бухгалтер,0.0,Основное место работы,1.0,Работа,Работа


In [7]:
df = pd.read_excel(path_to_data + months[1] + '.xlsx')

In [9]:
df['stv'] = df.apply(lambda row: to_double(row, 'stv'), axis=1)

In [12]:
day_types = ['Основное место работы', 'Внутреннее совместительство', 'Внешнее совместительство']

In [11]:
df.groupby('type').count()

,inn,snils,sum,job,day,stv,status,status_pref
type,,,,,,,,
Внешнее совместительство,7646,7646,7603,7646,7646,7646,7646,7646
Внутреннее совместительство,24087,24087,24062,24087,24087,24087,24087,24087
Основное место работы,165285,165285,165108,165285,165285,165285,165282,165282
Подработка,34,34,25,34,34,34,0,0


In [15]:
df.groupby('type').nunique()

,inn,snils,sum,job,day,stv,status,status_pref
type,,,,,,,,
Внешнее совместительство,550,7263,6357,134,2,170,13,13
Внутреннее совместительство,397,20916,14273,114,2,388,18,18
Основное место работы,596,165129,140022,235,2,581,22,22
Подработка,1,22,15,5,1,7,0,0


In [13]:
df1 = df[df.type.isin(day_types[:2])]

In [19]:
df1.nunique()

inn               596
snils          165276
sum            153680
job               235
day                 2
type                2
stv               802
status             22
status_pref        22
dtype: int64

In [18]:
df1.groupby('type').nunique()

,inn,snils,sum,job,day,stv,status,status_pref
type,,,,,,,,
Внутреннее совместительство,397,20916,14273,114,2,388,18,18
Основное место работы,596,165129,140022,235,2,581,22,22


In [27]:
a = df1.groupby(['snils']).nunique()

In [25]:
df[df.snils == '001-703-734 92']

,inn,snils,sum,job,day,type,stv,status,status_pref
132269,5003021495,001-703-734 92,7262.00,Помощник воспитателя,1,Внутреннее совместительство,0.25,Работа,Работа
132270,5003021495,001-703-734 92,63327.99,Помощник воспитателя,1,Основное место работы,1.00,Работа,Работа


In [32]:
osn = list(df1[df1.type == day_types[0]]['snils'].unique())

In [34]:
len(osn)

165129

In [35]:
df1[(df1.type == day_types[1]) & (df1.snils.isin(osn) == False)]

,inn,snils,sum,job,day,type,stv,status,status_pref
344,7720016173,038-746-851 00,40579.82,Учитель,1,Внутреннее совместительство,0.50,Работа,Работа
2515,7717043346,023-354-881 36,36772.96,Старший преподаватель,0,Внутреннее совместительство,0.50,Отсутствие по невыясненным причинам,Отсутствие по невыясненным причинам
4776,7717043346,166-792-511 10,151283.32,Ассистент,0,Внутреннее совместительство,0.50,Отпуск по беременности и родам,Отпуск по беременности и родам
5195,7717043346,008-397-347 70,86000.00,Директор,1,Внутреннее совместительство,1.00,Работа,Работа
13866,7723864492,027-152-812 31,5461.95,Помощник воспитателя,0,Внутреннее совместительство,0.25,Работа,Работа
...,...,...,...,...,...,...,...,...,...
187693,7708235066,110-438-967 33,24166.67,Преподаватель,0,Внутреннее совместительство,0.50,Работа,Работа
189680,7716237691,139-434-866 92,65000.00,Методист,1,Внутреннее совместительство,0.50,Работа,Работа
194623,7743085744,160-412-393 23,10967.06,Педагог дополнительного образования,1,Внутреннее совместительство,0.16,Работа,Работа
194633,7743085744,163-173-833 63,11652.44,Педагог дополнительного образования,1,Внутреннее совместительство,0.17,Работа,Работа


In [29]:
a[a['inn'] > 1]['inn'].sum()

356

In [16]:
df2 = df1[['inn', 'snils', 'sum']].groupby(['inn', 'snils']).sum()

In [17]:
df2

sum
inn        snils                    
5003021368 001-171-362 75   41289.47
           001-191-281 85  139753.92
           001-244-107 65  109541.13
           001-263-234 77   81178.00
           001-322-594 80   75815.00
...                              ...
9718071371 199-418-719 35  131067.00
           203-134-432 96   50000.00
           205-412-630 13  199010.00
           205-678-005 60   24504.00
           207-424-489 53  120896.47

[165454 rows x 1 columns]

In [58]:
b = pd.merge(df2.reset_index(), df1[(df1.status != 'Увольнение')], how='left', on=['inn', 'snils']).drop(columns='sum_y').rename(columns={'sum_x': 'sum'})

In [59]:
b = b[b.type == day_types[0]]

In [60]:
b

,inn,snils,sum,job,day,type,stv,status,status_pref
0,5003021368,001-171-362 75,41289.47,Помощник воспитателя,0.0,Основное место работы,1.0,Болезнь,Болезнь
1,5003021368,001-191-281 85,139753.92,Учитель,1.0,Основное место работы,1.0,Работа,Работа
2,5003021368,001-244-107 65,109541.13,Учитель,1.0,Основное место работы,1.0,Работа,Работа
3,5003021368,001-263-234 77,81178.00,Воспитатель,1.0,Основное место работы,1.0,Работа,Работа
4,5003021368,001-322-594 80,75815.00,Специалист,1.0,Основное место работы,1.0,Работа,Работа
...,...,...,...,...,...,...,...,...,...
189354,9718071371,199-418-719 35,131067.00,Бухгалтер,1.0,Основное место работы,1.0,Работа,Работа
189355,9718071371,203-134-432 96,50000.00,Учитель,0.0,Основное место работы,1.0,Работа,Работа
189356,9718071371,205-412-630 13,199010.00,Начальник группы,1.0,Основное место работы,1.0,Работа,Работа
189357,9718071371,205-678-005 60,24504.00,Бухгалтер,0.0,Основное место работы,1.0,Работа,Работа


In [49]:
c = b[b.duplicated(['inn', 'snils'], keep=False)]

In [50]:
c[c.type == day_types[1]]

,inn,snils,sum_x,sum_y,job,day,type,stv,status,status_pref
964,5003021495,001-703-734 92,70589.99,7262.00,Помощник воспитателя,1.0,Внутреннее совместительство,0.250,Работа,Работа
1001,5003021495,010-374-085 98,76830.00,10893.00,Помощник воспитателя,1.0,Внутреннее совместительство,0.375,Работа,Работа
1021,5003021495,014-158-355 22,73312.50,7262.00,Помощник воспитателя,1.0,Внутреннее совместительство,0.250,Работа,Работа
1025,5003021495,014-904-309 23,143620.94,33840.21,Педагог-психолог,0.0,Внутреннее совместительство,1.000,Дополнительный отпуск,Дополнительный отпуск
1084,5003021495,022-601-478 03,82256.68,7262.00,Помощник воспитателя,1.0,Внутреннее совместительство,0.250,Работа,Работа
...,...,...,...,...,...,...,...,...,...,...
187961,9715206976,204-248-388 42,115000.00,27500.00,Учитель-логопед,1.0,Внутреннее совместительство,0.500,Работа,Работа
187962,9715206976,204-248-388 42,115000.00,2000.00,Педагог дополнительного образования,1.0,Внутреннее совместительство,0.100,Работа,Работа
187969,9715206976,210-512-230 81,126061.78,14700.00,Педагог дополнительного образования,1.0,Внутреннее совместительство,0.400,Работа,Работа
187970,9715206976,210-512-230 81,126061.78,15791.67,Педагог дополнительного образования,0.0,Внутреннее совместительство,0.300,Работа,Работа


In [46]:
b[b.duplicated('snils', keep=False)]

,inn,snils,sum_x,sum_y,job,day,type,stv,status,status_pref
435,5003021368,117-804-650 57,164644.13,164644.13,Учитель,1.0,Основное место работы,1.00,Работа,Работа
702,5003021368,152-874-125 73,148787.93,148787.93,Учитель,1.0,Основное место работы,1.00,Работа,Работа
740,5003021368,158-617-041 82,155985.49,155985.49,Учитель,1.0,Основное место работы,1.00,Работа,Работа
763,5003021368,162-506-528 51,144186.46,144186.46,Учитель,1.0,Основное место работы,1.00,Работа,Работа
1381,5003021495,080-146-807 44,19878.66,19878.66,Педагог дополнительного образования,1.0,Основное место работы,0.33,Работа,Работа
...,...,...,...,...,...,...,...,...,...,...
165425,9718071371,181-257-254 64,238333.34,238333.34,Учитель,0.0,Основное место работы,1.00,Работа,Работа
165428,9718071371,183-108-423 50,400000.00,400000.00,Учитель,0.0,Основное место работы,1.00,Работа,Работа
165433,9718071371,185-940-990 25,210000.00,210000.00,Учитель,0.0,Основное место работы,1.00,Работа,Работа
165449,9718071371,198-731-766 35,440000.00,440000.00,Учитель,0.0,Основное место работы,1.00,Работа,Работа


In [10]:
df

,inn,snils,sum,job,day,type,stv,status,status_pref
0,7734408766,001-879-913 57,119742.67,Преподаватель,1,Основное место работы,1.0,Работа,Работа
1,7734408766,001-896-420 39,142964.10,Преподаватель,1,Основное место работы,1.0,Работа,Работа
2,7734408766,002-050-383 67,68007.79,Преподаватель,1,Основное место работы,1.0,Работа,Работа
3,7734408766,002-065-271 85,91097.30,Преподаватель,1,Основное место работы,1.0,Работа,Работа
4,7734408766,002-845-525 25,114596.10,Преподаватель,1,Основное место работы,1.0,Работа,Работа
...,...,...,...,...,...,...,...,...,...
199468,7722069597,186-373-917 14,67494.74,Специалист по интернет-маркетингу,0,Основное место работы,1.0,Работа,Работа
199469,7722069597,187-703-552 01,62968.42,Воспитатель,0,Основное место работы,1.0,Болезнь,Болезнь
199470,7722069597,198-391-256 24,91973.89,Воспитатель,1,Основное место работы,1.0,Работа,Работа
199471,7722069597,202-693-820 52,73130.37,Учитель,0,Основное место работы,1.0,Болезнь,Болезнь
